In [2]:
import pandas as pd
import logging 
import numpy as np
import re 
from IPython.display import display
from sqlalchemy import MetaData, Table
from database_utils import DatabaseConnector
from data_extraction import DataExtractor

In [ ]:
#still working on weight 

In [7]:
 #LOGGING start of method 
print('started convert_weights_to_kg')

#creating instance of dataextractor 
instance = DataExtractor()
    
#retrieving the data from the stores API
df = instance.extract_from_s3('s3://data-handling-public/products.csv') 
                
        


started convert_weights_to_kg
extract_from_s3 is working


In [8]:
#making a back up of the dataframe 
df_backup = df.copy()

In [24]:
# run this to reset the df 
df = df_backup.copy()

In [19]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1853 non-null   int64 
 1   product_name   1849 non-null   object
 2   product_price  1849 non-null   object
 3   weight         1849 non-null   object
 4   category       1849 non-null   object
 5   EAN            1849 non-null   object
 6   date_added     1849 non-null   object
 7   uuid           1849 non-null   object
 8   removed        1849 non-null   object
 9   product_code   1849 non-null   object
dtypes: int64(1), object(9)
memory usage: 144.9+ KB


None

In [27]:
display(df['removed'].unique())

array(['Still_avaliable', 'Removed', nan, 'T3QRRH7SRP', 'BPSADIOQOK',
       'H5N71TV8AY'], dtype=object)

In [33]:
nan_rows = df.loc[df['removed'] == 'nan']

display(nan_rows) 
#na_rows = df.loc[df['lat'] == 'N/A']

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code


In [34]:
df = df_backup.copy()


num_rows = df.shape[0]
display(f"Number of rows before cleaning: {num_rows}")

# Define the regex pattern to match numbers and letters
weight_pattern = re.compile(r'([0-9.]+)([a-zA-Z]+)')

# Conversion factors to kg
conversion_factors = {
    'kg': 1,
    'g': 0.001,
    'oz': 0.0283495231,
    'ml': 0.001  # Assuming ml is equivalent to grams for water-based products
}

def convert_to_kg(weight):
    if pd.isna(weight):
        return None  # Handle NaN values
    match = weight_pattern.match(str(weight))
    if match:
        number = float(match.group(1))  # Extract the numeric part
        unit = match.group(2).lower()  # Extract the unit part
        return number * conversion_factors.get(unit, 0)  # Convert to kg
    return None  # Handle cases where regex does not match

# Apply the conversion to the 'weight' column
df['weight_in_kg'] = df['weight'].apply(convert_to_kg)

num_rows = df.shape[0]
display(f"Number of rows after conversion: {num_rows}")

# Correct the misspelled value
#df['removed'] = df['removed'].replace('Still_avaliable', 'Still_available')
df['removed'].str.replace('Still_avaliable', 'Still_available') 


# Filter the DataFrame to keep only 'Still_available' or 'Removed'
valid_values = ['Still_available', 'Removed']
df = df[df['removed'].isin(valid_values)]

num_rows = df.shape[0]
display(f"Number of rows after cleaning of removed column: {num_rows}")


# Capture rows with NaN values before dropping them
rows_with_na = df[df.isna().any(axis=1)]
display("Rows with NaN values:")
display(rows_with_na)

# Drop rows with NaN values
df = df.dropna(axis=0)

num_rows = df.shape[0]
display(f"Number of rows after dropna: {num_rows}")

# Define the regex pattern for the 'category' column
category_pattern = re.compile(r'^[a-zA-Z\-]+$')

# Capture rows that will be dropped by the category filter
rows_dropped_by_category = df[~df['category'].apply(lambda x: bool(category_pattern.match(str(x))))]

display("Rows that will be dropped by category filter:")
display(rows_dropped_by_category)

# Filter rows based on the category pattern
df = df[df['category'].apply(lambda x: bool(category_pattern.match(str(x))))]

num_rows = df.shape[0]
display(f"Number of rows after category filter: {num_rows}")




'Number of rows before cleaning: 1853'

'Number of rows after conversion: 1853'

'Number of rows after cleaning of removed column: 94'

'Rows with NaN values:'

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code,weight_in_kg
1676,1676,Nescafe Dolce Gusto - Latte Machiato 16pk,£3.69,16 x 10g,food-and-drink,8057338051145,2005-08-23,07aa6951-d029-46ed-8a63-12e3ba9db4b7,Removed,C4-9106468S,NaN
1692,1692,Heinz Tomato Soup 4 x 400g,£3.39,4 x 400g,food-and-drink,1685057010180,2020-10-10,d4d54e79-cd6d-4f4b-9bee-4058403299de,Removed,G8-4206489Y,NaN


'Number of rows after dropna: 92'

'Rows that will be dropped by category filter:'

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code,weight_in_kg


'Number of rows after category filter: 92'